<a href="https://colab.research.google.com/github/surajjorwar/FitnessTracker/blob/main/make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read single CSV file


In [66]:
import pandas as pd
from glob import glob

In [67]:
single_csv = pd.read_csv("/content/drive/MyDrive/FitnessTracker/FitnessTracker/data-science-template-main/data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
single_csv.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g)
0,1547219408431,2019-01-11T16:10:08.431,0.00,0.01,0.964,-0.087
1,1547219408511,2019-01-11T16:10:08.511,0.08,0.00,0.961,-0.069


# List all data in data/raw/MetaMotion


In [68]:
files = glob('/content/drive/MyDrive/FitnessTracker/FitnessTracker/data-science-template-main/data/raw/MetaMotion/*csv')
len(files)

187

In [69]:
df = pd.read_csv(files[0])
df.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s)
0,1547579347087,2019-01-15T20:09:07.087,0.00,4.451,-1.159,-0.061
1,1547579347127,2019-01-15T20:09:07.127,0.04,3.902,-1.951,-1.220




# Extract features from filename

In [70]:
#extract feature from file
data_Path = '/content/drive/MyDrive/FitnessTracker/FitnessTracker/data-science-template-main/data/raw/MetaMotion/'

f = files[2].replace(data_Path,'')
participant = f.split("-")[0]
label = f.split("-")[1]
category = f.split("-")[2].rstrip("13MetaWear_2019")

In [71]:
df['participant'] = participant
df['label'] = label
df['category'] = category


df['participant'] = participant
df['label'] = label
df['category'] = category

In [72]:
df.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category
0,1547579347087,2019-01-15T20:09:07.087,0.00,4.451,-1.159,-0.061,E,rest,sitting
1,1547579347127,2019-01-15T20:09:07.127,0.04,3.902,-1.951,-1.220,E,rest,sitting




# Read all files


In [73]:
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

acc_set = 1
gyr_set = 1

for f in files:
  df = pd.read_csv(f)

  f = f.replace(data_Path,'')
  participant = f.split("-")[0]
  label = f.split("-")[1]
  category = f.split("-")[2].rstrip("13MetaWear_2019")


  df['participant'] = participant
  df['label'] = label
  df['category'] = category

  if "Accelerometer" in f:
    df['set'] = acc_set
    acc_set += 1
    acc_df = pd.concat([acc_df,df])

  if "Gyroscope" in f:
    df['set'] = gyr_set
    gyr_set += 1
    gyr_df = pd.concat([gyr_df,df])


In [74]:
acc_df.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g),participant,label,category,set
0,1548004946030,2019-01-20 18:22:26.029,0.00,0.979,-0.066,-0.188,E,rest,sitting,1
1,1548004946110,2019-01-20 18:22:26.109,0.08,0.983,-0.071,-0.185,E,rest,sitting,1


# Working with datetimes


In [75]:
pd.to_datetime(df['epoch (ms)'],unit= 'ms')

acc_df.index = pd.to_datetime(acc_df['epoch (ms)'],unit= 'ms')
gyr_df.index = pd.to_datetime(gyr_df['epoch (ms)'],unit= 'ms')

del acc_df["epoch (ms)"]
del acc_df['time (01:00)']
del acc_df['elapsed (s)']


del gyr_df["epoch (ms)"]
del gyr_df['time (01:00)']
del gyr_df['elapsed (s)']

# Turn into function

In [76]:
def read_data_file(files):
  acc_df = pd.DataFrame()
  gyr_df = pd.DataFrame()
  acc_set = 1
  gyr_set = 1


  for f in files:

        df = pd.read_csv(f)
        f = f.replace(data_Path,'')
        participant = f.split("-")[0]
        label = f.split("-")[1]
        category = f.split("-")[2].rstrip("123").rstrip("13MetaWear_2019")


        df['participant'] = participant
        df['label'] = label
        df['category'] = category

        if "Accelerometer" in f:
          df['set'] = acc_set
          acc_set += 1
          acc_df = pd.concat([acc_df,df])

        if "Gyroscope" in f:
          df['set'] = gyr_set
          gyr_set += 1
          gyr_df = pd.concat([gyr_df,df])

  acc_df.index = pd.to_datetime(acc_df['epoch (ms)'],unit= 'ms')
  gyr_df.index = pd.to_datetime(gyr_df['epoch (ms)'],unit= 'ms')

  del acc_df["epoch (ms)"]
  del acc_df['time (01:00)']
  del acc_df['elapsed (s)']


  del gyr_df["epoch (ms)"]
  del gyr_df['time (01:00)']
  del gyr_df['elapsed (s)']

  return acc_df,gyr_df
acc_df,gyr_df = read_data_file(files)


# Merging datasets

In [77]:
data_merged = pd.concat([acc_df.iloc[:,:3],gyr_df],axis = 1)

In [78]:
data_merged['set'].unique()

array([71., nan, 69., 39., 86., 78., 11., 74., 48., 13., 33., 81., 53.,
       28., 77., 23., 16., 64., 59., 56., 88., 60.,  3., 12., 21., 72.,
       83., 84., 20., 10., 61., 22., 40., 24., 35.,  2., 76., 87., 91.,
       37., 63.,  7.,  5., 46., 43., 80., 85., 19., 79., 54., 90., 62.,
        1., 32., 66., 55., 26., 27., 36., 52., 75., 17., 30., 42., 15.,
       58.,  6., 45., 82., 93., 14., 89., 18., 50.,  4., 49., 31., 29.,
        9., 44., 92., 65., 73., 57., 70., 41., 67., 34., 25., 51., 47.,
       38.,  8., 68.])

In [79]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 69677 entries, 2019-01-11 15:08:04.950000 to 2019-01-20 17:35:13.702000
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   x-axis (g)      23578 non-null  float64
 1   y-axis (g)      23578 non-null  float64
 2   z-axis (g)      23578 non-null  float64
 3   x-axis (deg/s)  47218 non-null  float64
 4   y-axis (deg/s)  47218 non-null  float64
 5   z-axis (deg/s)  47218 non-null  float64
 6   participant     47218 non-null  object 
 7   label           47218 non-null  object 
 8   category        47218 non-null  object 
 9   set             47218 non-null  float64
dtypes: float64(7), object(3)
memory usage: 5.8+ MB


# column rename

In [80]:
data_merged.columns = [
    'acc_x',
    'acc_y',
    'acc_z',
    'gyr_x',
    'gyr_y',
    'gyr_z',
    'participant' ,
    'label' ,
    'category' ,
    'set'
]

# Resample data (frequency conversion)



In [81]:
sampling = {
    'acc_x': 'mean',
    'acc_y': 'mean',
    'acc_z': 'mean',
    'gyr_x': 'mean',
    'gyr_y': 'mean',
    'gyr_z': 'mean',
    'participant' : "last",
    'label': "last" ,
    'category' : "last",
    'set': "last"
}

In [82]:
#apply mean for numerical and last for categorial features set is numerical and for this last is applied
data_merged[:1000].resample(rule = '200ms').apply(sampling)

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:04.800,NaN,NaN,NaN,-9.69550,-1.7985,4.5735,B,bench,heavy,71.0
2019-01-11 15:08:05.000,NaN,NaN,NaN,-2.01220,-0.4270,-1.6218,B,bench,heavy,71.0
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.89040,2.4392,0.9388,B,bench,heavy,71.0
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.68260,-0.8904,2.1708,B,bench,heavy,71.0
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.56080,-0.2560,-1.4146,B,bench,heavy,71.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-11 15:10:21.600,-0.174000,1.092667,-0.153333,-4.96360,-16.4026,21.9026,A,bench,heavy,69.0
2019-01-11 15:10:21.800,-0.219500,1.402500,-0.109000,-7.52440,18.6706,-22.6828,A,bench,heavy,69.0
2019-01-11 15:10:22.000,-0.192000,0.871667,-0.155667,2.60980,8.7070,-14.8658,A,bench,heavy,69.0


In [83]:
# groups by days and gives   dataframe for each day
days = [g for n,g in data_merged.groupby(pd.Grouper(freq='D')) ]

In [84]:
 #merges the data frame of all days
data_resampled = pd.concat([df.resample(rule = '200ms').apply(sampling).dropna() for df in days])

In [85]:
data_resampled


,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,71.0
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,71.0
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,71.0
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,71.0
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,71.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,68.0
2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,68.0
2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,68.0


In [86]:
data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   float64
dtypes: float64(7), object(3)
memory usage: 774.2+ KB


In [87]:
data_resampled['set'] = data_resampled['set'].astype(int)


# Export dataset

In [88]:
data_resampled.to_pickle('/content/drive/MyDrive/FitnessTracker/FitnessTracker/data-science-template-main/data/interim/01_data_processed.pkl')